In [1]:
import pandas as pd
from sklearn import *
df = pd.read_csv('prepped_churn_data.csv', index_col='customerID')
df = df.drop('Unnamed: 0', axis=1)
df = df.drop('tenure_total_charges_ratio', axis=1)
df

,tenure,PhoneService,Contract,PaymentMethod,MonthlyCharges,TotalCharges,Churn
customerID,,,,,,,
7590-VHVEG,1,0,0,0,29.85,29.85,0
5575-GNVDE,34,1,1,1,56.95,1889.50,0
3668-QPYBK,2,1,0,1,53.85,108.15,1
7795-CFOCW,45,0,1,2,42.30,1840.75,0
9237-HQITU,2,1,0,0,70.70,151.65,1
...,...,...,...,...,...,...,...
6840-RESVB,24,1,1,1,84.80,1990.50,0
2234-XADUH,72,1,1,3,103.20,7362.90,0
4801-JZAZL,11,0,0,0,29.60,346.45,0


In [2]:
from pycaret.classification import setup, compare_models, predict_model, save_model, load_model
#from pycaret.classification import *

In [3]:
#automl = setup(df, target='Churn', fold_shuffle=True)
automl = setup(df, target='Churn', fold_shuffle=True)

,Description,Value
0,session_id,660
1,Target,Churn
2,Target Type,Binary
3,Label Encoded,"0: 0, 1: 1"
4,Original Data,"(7032, 7)"
5,Missing Values,False
6,Numeric Features,3
7,Categorical Features,3
8,Ordinal Features,False
9,High Cardinality Features,False


In [4]:
automl[3]

False

In [5]:
best_model = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.7970,0.8377,0.4873,0.6566,0.5581,0.4305,0.4392,0.0450
ridge,Ridge Classifier,0.7962,0.0000,0.4697,0.6639,0.5494,0.4228,0.4338,0.0050
catboost,CatBoost Classifier,0.7962,0.8352,0.4935,0.6523,0.5603,0.4314,0.4393,0.4880
lr,Logistic Regression,0.7936,0.8321,0.5173,0.6368,0.5701,0.4363,0.4408,0.2900
lightgbm,Light Gradient Boosting Machine,0.7936,0.8247,0.5111,0.6391,0.5656,0.4330,0.4390,0.0290
lda,Linear Discriminant Analysis,0.7920,0.8252,0.5219,0.6309,0.5699,0.4345,0.4388,0.0050
ada,Ada Boost Classifier,0.7909,0.8387,0.4827,0.6376,0.5484,0.4162,0.4235,0.0220
xgboost,Extreme Gradient Boosting,0.7796,0.8115,0.4980,0.6004,0.5434,0.4001,0.4037,0.1910
rf,Random Forest Classifier,0.7731,0.7965,0.4965,0.5848,0.5351,0.3869,0.3902,0.0570
knn,K Neighbors Classifier,0.7682,0.7533,0.4559,0.5790,0.5087,0.3603,0.3654,0.1890


In [6]:
best_model

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=660, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [7]:
df.iloc[-2:-1].shape

(1, 7)

In [8]:
df.iloc[-1].shape

(7,)

In [9]:
predict_model(best_model, df.iloc[-2:-1])

,tenure,PhoneService,Contract,PaymentMethod,MonthlyCharges,TotalCharges,Churn,Label,Score
customerID,,,,,,,,,
8361-LTMKD,4,1,0,1,74.4,306.6,1,1,0.6288


In [10]:
save_model(best_model, 'GBC')

Transformation Pipeline and Model Succesfully Saved


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=[],
                                       display_types=True, features_todrop=[],
                                       id_columns=[],
                                       ml_usecase='classification',
                                       numerical_features=[], target='Churn',
                                       time_features=[])),
                 ('imputer',
                  Simple_Imputer(categorical_strategy='not_available',
                                 fill_value_categorical=None,
                                 fill_value_numerical=None,
                                 numeric_strate...
                                             learning_rate=0.1, loss='deviance',
                                             max_depth=3, max_features=None,
                                             max_leaf_nodes=None,
                                             min_i

In [11]:
import pickle

with open('GBC_model.pk', 'wb') as f:
    pickle.dump(best_model, f)

In [12]:
with open('GBC_model.pk', 'rb') as f:
    loaded_model = pickle.load(f)

In [13]:
new_data = df.iloc[-2:-1].copy()
new_data.drop('Churn', axis=1, inplace=True)
loaded_model.predict(new_data)

ValueError: Number of features of the model must match the input. Model n_features is 11 and input n_features is 6 

In [14]:
loaded_gbc = load_model('GBC')

Transformation Pipeline and Model Successfully Loaded


In [15]:
predict_model(loaded_gbc, new_data)

,tenure,PhoneService,Contract,PaymentMethod,MonthlyCharges,TotalCharges,Label,Score
customerID,,,,,,,,
8361-LTMKD,4,1,0,1,74.4,306.6,1,0.6288


In [16]:
from IPython.display import Code

Code('predict_churn.py')

import pandas as pd
from pycaret.classification import predict_model, load_model

def load_data(filepath):
    """
    Loads churn data into a DataFrame from a string filepath.
    """
    df = pd.read_csv(filepath, index_col='customerID')
    return df


def make_predictions(df):
    """
    Uses the pycaret best model to make predictions on data in the df dataframe.
    """
    model = load_model('GBC')
    predictions = predict_model(model, data=df)
    predictions.rename({'Label': 'Churn_prediction'}, axis=1, inplace=True)
    predictions['Churn_prediction'].replace({1: 'Churn', 0: 'No churn'},
                                            inplace=True)
    return predictions['Churn_prediction']


if __name__ == "__main__":
    df = load_data('new_churn_data.csv')
    predictions = make_predictions(df)
    print('predictions:')
    print(predictions)

In [17]:
%run predict_churn.py

Transformation Pipeline and Model Successfully Loaded
predictions:
customerID
9305-CKSKC       Churn
1452-KNGVK    No churn
6723-OKKJM    No churn
7832-POPKP    No churn
6348-TACGU    No churn
Name: Churn_prediction, dtype: object


Brief Analysis
- use pycaret to find an ML algorithm that performs best on the data. Choose a metric you think is best to use for finding the best model; by default, it is accuracy but it could be AUC, precision, recall, etc. The week 3 FTE has some information on these different metrics.

For a metric, I focused on the accuracy. I did find that three different models produced the highest accuracy after running three different times. Logistic Regression, Catboost Classifier, and Gradient Boosting Classifier each showed the highest accuracy on different runs. I went with the second model, Gradient Boosting Classifier. 

- save the model to disk

Model is saved as .pkl/.pk files.

- create a Python script/file/module with a function that takes a pandas dataframe as an input and returns the probability of churn for each row in the dataframe your Python file/function should print out the predictions for new data (new_churn_data.csv)the true values for the new data are [1, 0, 0, 1, 0] if you're interested. test your Python module and function with the new data, new_churn_data.csv

'predict_churn.py' is used to predict the churn for the new data set. This file is using the GBC model to produce the predictions. 

- write a short summary of the process and results at the end of this notebook
Since the new_churn_data file does not contain two of the columns that the prepped_churn_data file contains. The 'Unnamed: 0' and 'tenure_total_charges_ratio' columns were causing errors, so I dropped them prior to generating a model. Additionally, it seemed that the best model changed on each run that was performed. The final model chosen, GBC, did not have a completely accurate prediction based on the truth data. Both the truth and model prediction show that row 1 of the new_churn_data is a churn candidate, and rows 2, 3, and 5 are not churn candidates. There is a mismatch for row 4.

- upload this Jupyter Notebook and Python file to a Github repository, and turn in a link to the repository in the week 5 assignment dropbox
Link to GitHub repository that contains this Jupyter notebook and Python file. https://github.com/averagetree/IntroDataScience/